# Imports

In [860]:
import pandas as pd
import numpy as np

# Phase 1
> 2022-2024; NA Listed Firms;

## Emissions Data

### Load

In [861]:
""" Read in the CSV """
raw_emissions_df = pd.read_csv(
    "phase1_emissions.csv", 
    dtype={"companyid": "str", "gvkey": "str"}, #identifiers
    parse_dates=["periodenddate"], 
)

raw_emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
0,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,6520204,NaN,Factory Mutual Insurance Company,United States
1,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,24951392,NaN,"Factory Mutual Insurance Company, Asset Manage...",United States
2,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,43964734,NaN,"Allendale Mutual Insurance Co, Asset Managemen...",United States
3,11489,2023,2023-12-31,4154.828402,12742.671094,176205.682880,6520204,NaN,Factory Mutual Insurance Company,United States
4,11489,2023,2023-12-31,4154.828402,12742.671094,176205.682880,24951392,NaN,"Factory Mutual Insurance Company, Asset Manage...",United States
...,...,...,...,...,...,...,...,...,...,...
1803283,118526590,2022,2022-12-31,485.540053,1085.798816,1022.548683,111952300,NaN,Coseva Soc Coop,Italy
1803284,118706040,2022,2022-12-31,205.995561,658.395477,1709.587413,1886594015,NaN,Solà Domingo S.A.,Spain
1803285,118918440,2022,2022-07-31,40.124678,26.875831,83.556928,1887792515,NaN,"Nihonsangyo Co.,Ltd.",Japan
1803286,118991426,2022,2022-12-31,59.719416,216.809138,129.375410,1888091056,NaN,CP Management Spólka Z Ograniczona Odpowiedzia...,Poland


### EDA (Can be skipped)

**Inspect the loaded datatypes**

In [679]:
raw_emissions_df.dtypes

institutionid             int64
fiscalyear                int64
periodenddate    datetime64[ns]
di_319413               float64
di_319414               float64
di_319415               float64
companyid                object
gvkey                    object
companyname              object
country                  object
dtype: object

**The company ids are extracted and written out to a file**

In [680]:
""" Utility functions regarding validity, uniqueness, and writing out 
unique and valid ids. """

def keep_valid(data, colname=None):
    """ Only keeps the rows/items from the given dataframe/array-like which 
    have valid values - for the specified column in the case of a dataframe.

    Args:
        data: Dataframe or array-like.
        colname: Name of the dataframe column whose valid values we are using to 
            filter.

    Returns:
        The dataframe/array-like with rows/items that have invalid values, w.r.t 
        the specified column if applicable, filtered out.
    """

    # Array whose values we are interested in
    col = data[colname] if colname is not None else data
    # Values are considered valid as long as they are not NaN
    return data[pd.notnull(col)]


def extract_unique(df, colname):
    """ Extract the unique and non-NaN values from a dataframe column.
    
    Args:
        df: Dataframe.
        colname: Column name w.r.t the dataframe.

    Returns:
        Unique and non-NaN column values.
    """

    return keep_valid(pd.unique(df[colname]))


def write_ids(df, idname, filename):
    """ Writes the unique, non-NaN instances of the indicated identifier, within the 
    indicated dataframe, on separate lines of a new file, whose filename should 
    be specified.

    Args:
        df: Dataframe.
        idname: Column name of the identifier with respect to the dataframe.
        filename: The name to use for the newly created file.
    """
    with open(filename, "w") as fh:
        for idval in extract_unique(df, idname):
            fh.write(f"{idval}\n")

In [4]:
""" Export CIQ company ids """
write_ids(raw_emissions_df, "companyid", "companyids.txt")

**The number of unique company ids and gvkeys are reported for this raw emissions data**

In [681]:
""" Utility function """
def report_unique(df, colname):
    """ Reports unique, non-NaN values of a dataframe column
    
    Args:
        df: Dataframe.
        colname: Column name.
    """

    print(f"Number of unique, non-null values of \'{colname}\': {len(extract_unique(df, colname))}")

In [682]:
print("-- Raw Emissions Data")
report_unique(raw_emissions_df, "institutionid")
report_unique(raw_emissions_df, "companyid")
report_unique(raw_emissions_df, "gvkey")

-- Raw Emissions Data
Number of unique, non-null values of 'institutionid': 1720932
Number of unique, non-null values of 'companyid': 1725359
Number of unique, non-null values of 'gvkey': 24304


**Check whether every gvkey entry has a corresponding company id (non-null -> non-null)**

In [684]:
def is_backed(df, src_colname, ref_colname):
    """ Check if one column is backed by another in a dataframe. This means that 
    when the first column is non-null, the second column cannot be null as 
    otherwise it would be backing the first column.

    Args:
        df: Dataframe.
        src_colname: Name of column that should be backed.
        ref_colname: Name of backing column.
    
    Returns:
        True/False depending on whether the first column is backed by the second
    """

    # check for 0 invalid cases for valid backing
    return (df[src_colname].notnull() & df[ref_colname].isnull()).sum() == 0

def report_backed(df, src_colname, ref_colname):
    """ Reports whether every non-null value of src is backed by a non-null 
    value of ref.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that should be backed.
        ref_colname: Name of backing column.
    """
    print(f"Is \'{src_colname}\' backed by \'{ref_colname}\': {'YES' if is_backed(df, src_colname, ref_colname) else 'NO'}")

In [685]:
print("-- Raw Emissions Data")
report_backed(
    raw_emissions_df, "gvkey", "companyid"
)
report_backed(
    raw_emissions_df, "companyid", "institutionid"
)

-- Raw Emissions Data
Is 'gvkey' backed by 'companyid': YES
Is 'companyid' backed by 'institutionid': YES


**Check for duplicates and null values in the 3 identifiers - companyid, gvkey and institutionid**

In [686]:
""" Utilty functions """
def report_null(df, colname):
    """ Indicate the number of nulls in the specified column in the 
    given dataframe.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that we are interested in.
    """
    print(f"Number of nulls in \'{colname}\': {df[colname].isnull().sum()}")

def report_dup(df, colname):
    """ Indicate the presence of duplicates in the specified column in the 
    given dataframe.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that we are interested in.
    """

    print(f"Are there duplicates in \'{colname}\': {'YES' if df[colname].duplicated().any() else 'NO'}")

In [687]:
print("-- Raw Emissions Data")
print("> Null checks")
report_null(raw_emissions_df, "companyid")
report_null(raw_emissions_df, "institutionid")
report_null(raw_emissions_df, "gvkey")
print("> Dup checks")
report_dup(raw_emissions_df, "companyid")
report_dup(raw_emissions_df, "institutionid")
report_dup(raw_emissions_df, "gvkey")

-- Raw Emissions Data
> Null checks
Number of nulls in 'companyid': 747
Number of nulls in 'institutionid': 0
Number of nulls in 'gvkey': 1769291
> Dup checks
Are there duplicates in 'companyid': YES
Are there duplicates in 'institutionid': YES
Are there duplicates in 'gvkey': YES


**Utilities for numerical value conflicts on grouping by a key. Check for one to one mapping conflicts between identifiers.**

In [688]:
""" Utility functions """

# aggregates series values, NaN if all equal, else string representation with space delimiter
# TODO: assumes no NaN in vals and also the existence of at least 1 element due 
# to the retrieval at index 0.
numer_conflict_aggr = lambda vals: np.nan if (vals.iloc[0] == vals).all() else " ".join(vals.astype(str))

def detect_numerical_conflicts(df, key, numer_cols):
    """ Detects numerical conflicts when grouping on a dataframe by the given 
    keys - filling in the multiple conflict values for inspection.
    
    Args:
        df: Dataframe.
        key: Key or keys that are used for grouping.
        numer_cols: Name of numerical columns which are checked for conflicts
    
    Returns:
        Dataframe with values as NaN if no conflicts, else the conflict values 
        delimited by spaces are filled in.
    """

    return df.groupby(key).agg(
        {
            numer_col : numer_conflict_aggr for numer_col in numer_cols
        }
    )

def one_to_one_conflict_aggr(vals):
    """ Aggregator for the 1t1 conflicts, NaN indicates valid as 1 mapped, else 
    the conflicting values of the second identifier are returned 
    (nothing or multiple).
    
    Args:
        vals: Series that is aggregated.
    Returns:
        NaN if valid, else the conflict values joined together in a string, 
        separated by spaces.
    """
    # simple short-circuit on apparent validity
    if vals.count() == 1:
        return np.nan
    # checking if there is conflict by there being no mapping
    if vals.count() == 0:
        return ""

    # otherwise, we have multiple entries (>1)
    vals = vals.dropna() # remove null bloat to leave just the entries

    if (vals.iloc[0] == vals).all(): # reduces to 1 actual mapping, hence valid
        return np.nan
    else: # otherwise, multiple actual mappings, which creates conflicts
        return " ".join(vals.astype(str))

def detect_1t1_conflicts(df, key, oth_id):
    """ Detects one to one mapping conflicts when grouping on a dataframe by the 
    first identifier and seeing how many of the second identifier this maps to.
    
    Args:
        df: Dataframe.
        key: Key for the first identifier.
        oth_id: Name of the column for the second identifier
    
    Returns:
        Dataframe with values as NaN if no conflicts, else the conflict values, 
        for the second identifier, delimited by spaces are filled in.
    """

    return df.groupby(key).agg(
        {
            oth_id : one_to_one_conflict_aggr, 
        }
    )

**Numerical value conflicts in environmental metrics when grouping by gvkey and 
fiscal year.**

In [689]:
raw_emissions_gvkey_numer_conflicts = detect_numerical_conflicts(
    raw_emissions_df, 
    ["gvkey", "fiscalyear", "periodenddate"], 
    ["di_319413", "di_319414", "di_319415"]
)

raw_emissions_gvkey_numer_conflicts

di_319413 di_319414 di_319415
gvkey  fiscalyear periodenddate                              
001004 2022       2022-05-31          NaN       NaN       NaN
001045 2022       2022-12-31          NaN       NaN       NaN
       2023       2023-12-31          NaN       NaN       NaN
001050 2022       2022-12-31          NaN       NaN       NaN
       2023       2023-12-31          NaN       NaN       NaN
...                                   ...       ...       ...
362683 2022       2022-03-31          NaN       NaN       NaN
362705 2022       2022-12-31          NaN       NaN       NaN
362758 2022       2022-03-31          NaN       NaN       NaN
362761 2022       2022-03-31          NaN       NaN       NaN
       2023       2023-03-31          NaN       NaN       NaN

[33896 rows x 3 columns]

In [690]:
raw_emissions_gvkey_numer_conflicts[
    pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319413"])
    | pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319414"])
    | pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319415"])
]

di_319413  \
gvkey  fiscalyear periodenddate                              
003413 2022       2022-12-31        2867127.0 51062759.241   
003897 2022       2022-12-31     27114550.031 25358937.262   
004094 2022       2022-12-31      810.06 1410.524 7995.579   
005180 2022       2022-12-31        636360.11 53101.342601   
005600 2022       2022-12-31      1080082.1763 4073314.057   
...                                                    ...   
275535 2022       2022-03-31          734.626 112.41037212   
       2023       2023-03-31          611.138 80.405872624   
289724 2022       2022-12-31          101.653 0.0053786782   
295786 2022       2022-12-31       134627.057 0.9207597207   
318474 2022       2022-12-31            916.0 626.35933659   

                                               di_319414  \
gvkey  fiscalyear periodenddate                            
003413 2022       2022-12-31      1018275.0 3933.0872579   
003897 2022       2022-12-31       197000.0 1953.2613298   
004094 2022       2022-12-31     7696.0 2956.39 9723.273   
005180 2022       2022-12-31       251038.0 32063.589498   
005600 2022       2022-12-31        61712.663909 73116.0   
...                                                  ...   
275535 2022       2022-03-31        4498.412 111.2256901   
       2023       2023-03-31       3819.836 79.558482923   
289724 2022       2022-12-31       1642.732 0.0869208615   
295786 2022       2022-12-31     129177.195 3.5451698548   
318474 2022       2022-12-31         4813.0 1649.4464065   

                                                      di_319415  
gvkey  fiscalyear periodenddate                                  
003413 2022       2022-12-31           1650402.398 2024652.4121  
003897 2022       2022-12-31           4025432.601 1005488.8192  
004094 2022       2022-12-31     197193.363 84877.747 39817.706  
005180 2022       2022-12-31           2436669.603 254265.62927  
005600 2022       2022-12-31            850262.94718 353150.506  
...                                                         ...  
275535 2022       2022-03-31              8751.123 2720.5732053  
       2023       2023-03-31              6836.914 1945.9953604  
289724 2022       2022-12-31              7071.309 0.3741598245  
295786 2022       2022-12-31           5009667.962 64.639735408  
318474 2022       2022-12-31            109611.983 3010.0308692  

[95 rows x 3 columns]

It is evident from the above that the gvkeys are mapping to multiple company ids in 
the same period and this is leading to multiple instances of the environment metrics

**Gvkey <-> Companyid Mapping**

- *gvkey -> company id*

In [691]:
raw_emissions_gvkey_companyid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "gvkey", "companyid"
)

raw_emissions_gvkey_companyid_1t1_conflicts

,companyid
gvkey,
001004,NaN
001045,NaN
001050,NaN
001075,NaN
001076,NaN
...,...
362620,NaN
362683,NaN
362705,NaN


In [692]:
raw_emissions_gvkey_companyid_1t1_conflicts[
    pd.notnull(raw_emissions_gvkey_companyid_1t1_conflicts["companyid"])
]

,companyid
gvkey,
002856,259777 259777 3103613 3103613
003413,263295 6020983 6020983
003897,266598 1067744 1067744
004094,528325 27755 386000 386000
005180,275442 275442 5546439
...,...
275535,109366051 109366051 881331 881331
275839,53336883 5482350
289724,215005509 215005509 30941174


Notice that gvkey can map to multiple companyids, but remember that it 
maps to at least 1 company id because it backed by companyid.

- *company id -> gvkey*

In [693]:
raw_emissions_companyid_gvkey_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "companyid", "gvkey"
)

raw_emissions_companyid_gvkey_1t1_conflicts

,gvkey
companyid,
100000307,
100013,NaN
1000212,
1000277,
10004497,NaN
...,...
99996472,
99996476,
99996998,


In [694]:
raw_emissions_companyid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"] == "")
]

,gvkey
companyid,
100000307,
1000212,
1000277,
10004521,
10005029,
...,...
99996472,
99996476,
99996998,


In [695]:
raw_emissions_companyid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"] != "")
]

,gvkey
companyid,
104422749,037090 184982
106683684,285220 316585
1067744,003897 065089 003897 065089
1073371,100557 220942 100557 220942
112732,024616 145471 024616 145471
...,...
9683016,220688 247655
983017,007824 145270 007824 145270
9833116,242985 321595


Notice that companyid can both map to no or multiple gvkeys

**Institutionid <-> Companyid Mapping**

- *companyid -> institutionid*

In [696]:
raw_emissions_companyid_institutionid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "companyid", "institutionid"
)

raw_emissions_companyid_institutionid_1t1_conflicts

,institutionid
companyid,
100000307,NaN
100013,NaN
1000212,NaN
1000277,NaN
10004497,NaN
...,...
99996472,NaN
99996476,NaN
99996998,NaN


In [697]:
raw_emissions_companyid_institutionid_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_institutionid_1t1_conflicts["institutionid"])
]

,institutionid
companyid,


Hence, we can see companyid maps to exactly one institutionid.

- *institutionid -> companyid*

In [698]:
raw_emissions_institutionid_companyid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "institutionid", "companyid"
)

raw_emissions_institutionid_companyid_1t1_conflicts

,companyid
institutionid,
11489,6520204 24951392 43964734 6520204 24951392 439...
11654,NaN
11679,8333444 24586834 8333444 24586834
11894,7925667 34873695 7925667 34873695
12062,NaN
...,...
118526590,NaN
118706040,NaN
118918440,NaN


In [699]:
raw_emissions_institutionid_companyid_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_companyid_1t1_conflicts["companyid"])
]

,companyid
institutionid,
11489,6520204 24951392 43964734 6520204 24951392 439...
11679,8333444 24586834 8333444 24586834
11894,7925667 34873695 7925667 34873695
13644,26465936 60478967 26465936 60478967
13959,246652 6167099 242354247 246652 6167099 242354247
...,...
109987465,
110365423,
111445983,


Here, we can see that while a company belongs to exactly one institution, an 
institution maps to many companys (none, one or potentially many).

**Gvkey <-> Institutionid Mapping**

- *gvkey -> institutionid*

In [700]:
raw_emissions_gvkey_institutionid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "gvkey", "institutionid"
)

raw_emissions_gvkey_institutionid_1t1_conflicts

,institutionid
gvkey,
001004,NaN
001045,NaN
001050,NaN
001075,NaN
001076,NaN
...,...
362620,NaN
362683,NaN
362705,NaN


In [701]:
raw_emissions_gvkey_institutionid_1t1_conflicts[
    pd.notnull(raw_emissions_gvkey_institutionid_1t1_conflicts["institutionid"])
]

,institutionid
gvkey,
002856,4057039 4057039 4057076 4057076
003413,4057041 4057080 4057080
003897,4057044 4057083 4057083
004094,108462 4021861 4388004 4388004
005180,4806213 4806213 5053995
...,...
275535,4265945 4265945 4326804 4326804
275839,4295672 6343205
289724,4772912 4772912 6393031


Noting that there are no null values in institutionid, this means that gvkey 
corresponds to at least one, and potentially many, institutions.

- *institutionid -> gvkey*

In [702]:
raw_emissions_institutionid_gvkey_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "institutionid", "gvkey"
)

raw_emissions_institutionid_gvkey_1t1_conflicts

,gvkey
institutionid,
11489,
11654,
11679,
11894,
12062,
...,...
118526590,
118706040,
118918440,


In [703]:
raw_emissions_institutionid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"] == "")
]

,gvkey
institutionid,
11489,
11654,
11679,
11894,
12062,
...,...
118526590,
118706040,
118918440,


In [704]:
raw_emissions_institutionid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"] != "")
]

,gvkey
institutionid,
100165,002001 002002
100259,004708 027665 004708 027665
100391,008119 017095 008119 017095
101674,005849 039571
103042,012124 027867
...,...
6618361,204440 247501
6626040,278266 340246
6932029,100787 326859


We can see that institutionids map to many gvkeys (can be zero, one or multiple)

**Determine the number of gvkey identifiers that exist across all fiscal years and also the number that exist in each fiscal year**

First check whether `fiscalyear` is ever null

In [705]:
print("Is fiscalyear ever null:")
print('YES' if raw_emissions_df["fiscalyear"].isnull().any() else 'NO')

Is fiscalyear ever null:
NO


Check if there are any duplicate fiscal year entries for gvkeys

In [706]:
raw_emissions_gvkey_fiscalyear_dup = raw_emissions_df.groupby("gvkey").agg(
    {
        "fiscalyear": lambda vals: vals.duplicated().any(),
    }
)

raw_emissions_gvkey_fiscalyear_dup[
    raw_emissions_gvkey_fiscalyear_dup["fiscalyear"]
]

,fiscalyear
gvkey,
002856,True
003413,True
003897,True
004094,True
005180,True
...,...
271134,True
275535,True
289724,True


We can see there are, and this is due to the duplication from the varying 
`companyid`'s and `institutionid`'s

### Data Preparation
> Mainly involves filtering

In [870]:
cid_gvkey_df = pd.read_csv("cid_gvkey_map.csv")

cid_gvkey_df

,companyid,gvkey,startdate,enddate,companyname
0,18511,210835,B,E,3i Group plc
1,18527,210418,B,E,ABB Ltd
2,18671,29751,B,E,Albemarle Corporation
3,18711,28349,B,E,The Allstate Corporation
4,18749,64768,B,E,"Amazon.com, Inc."
...,...,...,...,...,...
24388,1849817514,361808,B,E,Kawan Renergy Berhad
24389,1855364409,358709,B,E,ELSA Solutions S.p.A.
24390,1855399529,358653,B,E,"SEIYU KOGYO Co.,Ltd."
24391,1859487646,359029,B,E,KET Inc.


Inspect the loaded datatypes

In [871]:
cid_gvkey_df.dtypes

companyid       int64
gvkey           int64
startdate      object
enddate        object
companyname    object
dtype: object

In [872]:
print("-- Null Values:")
cid_gvkey_df.isnull().sum()

-- Null Values:


companyid      0
gvkey          0
startdate      0
enddate        0
companyname    0
dtype: int64

Filter the mappings to just those that are one-to-one, and keep note of these.

In [873]:
cid_gvkey_1t1_df = cid_gvkey_df[(cid_gvkey_df["startdate"] == "B") & (cid_gvkey_df["enddate"] == "E")]

cid_gvkey_1t1_df

,companyid,gvkey,startdate,enddate,companyname
0,18511,210835,B,E,3i Group plc
1,18527,210418,B,E,ABB Ltd
2,18671,29751,B,E,Albemarle Corporation
3,18711,28349,B,E,The Allstate Corporation
4,18749,64768,B,E,"Amazon.com, Inc."
...,...,...,...,...,...
24388,1849817514,361808,B,E,Kawan Renergy Berhad
24389,1855364409,358709,B,E,ELSA Solutions S.p.A.
24390,1855399529,358653,B,E,"SEIYU KOGYO Co.,Ltd."
24391,1859487646,359029,B,E,KET Inc.


*Another approach for filtering mappings is considering where gvkey is not 
duplicated, as opposed to mappings that exist for all of time. This alternative 
approach may however lead to cases where the same company exists under 
different gvkeys (same companyid corresponds to multiple gvkeys). This was not 
chosen in our case.*

Verify that these mappings are indeed one-to-one.

In [874]:
print(f"Gvkey duplicates: {cid_gvkey_1t1_df['gvkey'].duplicated().any()}")
print(f"Companyid duplicates: {cid_gvkey_1t1_df['companyid'].duplicated().any()}")

Gvkey duplicates: False
Companyid duplicates: False


The lack of duplicates on both sides indicates that the mappings are indeed one to one.

>Notice filtering to one-to-one mappings reduces the number of unique gvkeys from ~24k to ~23k in the join table.

**Filter the main environment table using this join table, remembering the following facts:**
The raw main environment table has:
- ~1.8 million rows
- ~1.7 million unique companyid/institutionid values
- ~24k unique gvkey values

From the main environment table, filter entries to just those with gvkeys.

In [875]:
emissions_df = raw_emissions_df[pd.notnull(raw_emissions_df["gvkey"])]

**Further filter to just the gvkeys that 1 to 1 map with companyid.** 
This should leave one entry per fiscal year for each gvkey. 

*Note some gvkeys may not have an entry for certain fiscal years because that data is simply missing.*

In [876]:
emissions_df = emissions_df[
    np.in1d(emissions_df["gvkey"], cid_gvkey_1t1_df["gvkey"].astype(str))
]

emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States
...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan


Here, we check whether the above statement of exactly one entry per gvkey, 
fiscal year is true (barring the exception of missing data).

In [877]:
emissions_gvkey_fiscalyear_entries = emissions_df.groupby(["gvkey", "fiscalyear"]).size()

emissions_gvkey_fiscalyear_entries

gvkey   fiscalyear
100001  2022          1
        2023          1
100006  2022          1
100010  2022          1
100012  2022          1
                     ..
362683  2022          1
362705  2022          1
362758  2022          1
362761  2022          1
        2023          1
Length: 26407, dtype: int64

In [878]:
emissions_gvkey_fiscalyear_entries[
    emissions_gvkey_fiscalyear_entries != 1
]

Series([], dtype: int64)

The empty return for a number of entries different than 1 shows that 
we have indeed made gvkey, fiscal year unique.


In [880]:
emissions_df.nunique()["gvkey"]

19352

We can see that this process has reduced the number of unique `gvkey`'s to 
~19k now.

In [881]:
emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States
...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan


**Perform QC with respect to null values**

In [882]:
emissions_df.dropna()

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States
...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan


We can see that the number of rows is not reduced, therefore there are no 
null values and the data quality has been verified.

**Write out the gvkeys for linking with fundamentals and returns data**

In [243]:
write_ids(emissions_df, "gvkey", "gvkeys.txt")

**Inspect the fiscal years and the year-month accounting ends which are associated with them**

Create an extra column to isolate the month and year from the entire `periodenddate`. 
This column will be called `periodend_ym` to denote that it just retains the 
year and month information.
- This new column will be of the `period[M]` type

In [883]:
emissions_df["periodend_ym"] = emissions_df['periodenddate'].dt.to_period('M')

print(f"New Column Type: {emissions_df['periodend_ym'].dtype}")
emissions_df

New Column Type: period[M]


,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country,periodend_ym
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2022-12
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2023-12
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States,2022-12
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States,2023-12
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States,2022-12
...,...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia,2022-10
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy,2022-12
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan,2022-09
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan,2022-03


Group by fiscal year and year-month ends to see which year-month ends are 
contained within each fiscal year.

In [885]:
emissions_df.groupby(["fiscalyear", "periodend_ym"]).size()

fiscalyear  periodend_ym
2021        2022-01            61
2022        2022-01           113
            2022-02           210
            2022-03          3438
            2022-04           110
            2022-05           101
            2022-06           897
            2022-07           119
            2022-08           123
            2022-09           354
            2022-10            79
            2022-11            57
            2022-12         13506
            2023-01            22
2023        2023-01            75
            2023-02           130
            2023-03          1786
            2023-04            72
            2023-05            53
            2023-06           532
            2023-07            62
            2023-08            76
            2023-09           186
            2023-10            38
            2023-11            20
            2023-12          4186
            2024-01             1
dtype: int64

**Finalise DataFrame**
- according to joining considerations

In [886]:
emissions_df = emissions_df.reset_index(drop=True)

emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country,periodend_ym
0,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2022-12
1,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2023-12
2,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States,2022-12
3,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States,2023-12
4,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States,2022-12
...,...,...,...,...,...,...,...,...,...,...,...
26402,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia,2022-10
26403,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy,2022-12
26404,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan,2022-09
26405,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan,2022-03


- todo: extract fundamentals data of the same fiscal years (also see the link between periodenddate and fiscal year)
- todo: market data of one year ahead from the fiscal/fundamentals data
  - or at least one month

**Phase 1 Context Awareness**

Number of US based companies out of the ~19k unique gvkeys

In [887]:
(emissions_df.groupby("gvkey").first()["country"] == "United States").sum()

809

## Currency Data
- currency exchange fluctuations, when taking USD as the base currency, cause FX returns that should be accounted for
- market value and other quantities must be converted into a common currency (USD) to facilitate comparisons

Load daily currency exchange rates to USD & GBP. Sort by origin currency and then date.

In [888]:
exrts_df = pd.read_csv(
    "phase1_exrts.csv", 
    parse_dates=["datadate"], 
)
exrts_df = exrts_df.sort_values(["curd", "datadate"])

exrts_df

,curd,datadate,exratd_toGBP,exratd_toUSD
0,AED,2021-12-01,0.204428,0.272257
154,AED,2021-12-02,0.204553,0.272261
308,AED,2021-12-03,0.205901,0.272242
462,AED,2021-12-04,0.205901,0.272242
616,AED,2021-12-05,0.205901,0.272242
...,...,...,...,...
151561,ZWL,2024-08-14,0.002350,0.003019
151714,ZWL,2024-08-15,0.002350,0.003023
151867,ZWL,2024-08-16,0.002350,0.003032
152020,ZWL,2024-08-17,0.002350,0.003032


In [889]:
exrts_df.dtypes

curd                    object
datadate        datetime64[ns]
exratd_toGBP           float64
exratd_toUSD           float64
dtype: object

Compute monthly currency rates.
- Based on the exchange rates at the end of the months i.e. the last ones.

In [890]:
exrts_df["data_ym"] = exrts_df["datadate"].dt.to_period('M')

exrts_df

,curd,datadate,exratd_toGBP,exratd_toUSD,data_ym
0,AED,2021-12-01,0.204428,0.272257,2021-12
154,AED,2021-12-02,0.204553,0.272261,2021-12
308,AED,2021-12-03,0.205901,0.272242,2021-12
462,AED,2021-12-04,0.205901,0.272242,2021-12
616,AED,2021-12-05,0.205901,0.272242,2021-12
...,...,...,...,...,...
151561,ZWL,2024-08-14,0.002350,0.003019,2024-08
151714,ZWL,2024-08-15,0.002350,0.003023,2024-08
151867,ZWL,2024-08-16,0.002350,0.003032,2024-08
152020,ZWL,2024-08-17,0.002350,0.003032,2024-08


In [891]:
m_exrts = exrts_df.groupby(
    ["curd", "data_ym"]
).last(
).drop(
    columns="datadate"
)

m_exrts

exratd_toGBP  exratd_toUSD
curd data_ym                            
AED  2021-12      0.201066      0.272263
     2022-01      0.202852      0.272248
     2022-02      0.202972      0.272266
     2022-03      0.207117      0.272255
     2022-04      0.216892      0.272264
...                    ...           ...
ZWL  2024-04      0.002350      0.002944
     2024-05      0.002350      0.002991
     2024-06      0.002350      0.002970
     2024-07      0.002350      0.003018
     2024-08      0.002350      0.003032

[5063 rows x 2 columns]

## Returns Data

### NA & Global - Loading and Preliminary Inspection

**NA**

In [892]:
na_returns_df = pd.read_csv(
    "phase1_returns.csv", 
    parse_dates=["datadate"], 
)

na_returns_df

,gvkey,iid,datadate,conm,ajexm,curcdm,prccm,trfm,cshom
0,100001,01,2022-02-28,ACCOR SA,1.0,USD,35.01,NaN,2.618560e+08
1,100001,01,2022-03-31,ACCOR SA,NaN,USD,NaN,1.0000,2.618560e+08
2,100001,01,2022-06-30,ACCOR SA,1.0,USD,26.45,NaN,2.629960e+08
3,100001,01,2022-07-31,ACCOR SA,NaN,USD,NaN,1.0000,2.629960e+08
4,100001,01,2023-07-31,ACCOR SA,1.0,USD,37.90,NaN,2.648940e+08
...,...,...,...,...,...,...,...,...,...
65256,353444,90,2024-04-30,HALEON PLC,1.0,USD,8.53,1.0253,9.132301e+09
65257,353444,90,2024-05-31,HALEON PLC,1.0,USD,8.44,1.0253,9.132301e+09
65258,353444,90,2024-06-30,HALEON PLC,1.0,USD,8.26,1.0253,9.132301e+09
65259,353444,90,2024-07-31,HALEON PLC,1.0,USD,9.15,1.0253,9.132301e+09


In [893]:
na_returns_df.dtypes

gvkey                int64
iid                 object
datadate    datetime64[ns]
conm                object
ajexm              float64
curcdm              object
prccm              float64
trfm               float64
cshom              float64
dtype: object

Number of unique companies in the raw NA return data

In [894]:
na_returns_df["gvkey"].nunique()

1856

**Global**

In [895]:
global_returns_df = pd.read_csv("phase2_returns.csv")

global_returns_df

,gvkey,datadate,ajexm,prccm,dvpsxm
0,100001,2021-12-31,1.0,28.45,0.0
1,100001,2022-01-31,1.0,32.36,0.0
2,100001,2022-02-28,1.0,30.59,0.0
3,100001,2022-03-31,1.0,29.23,0.0
4,100001,2022-04-30,1.0,31.53,0.0
...,...,...,...,...,...
727700,362097,2024-04-30,1.0,NaN,0.0
727701,362097,2024-05-31,1.0,NaN,0.0
727702,362097,2024-06-30,1.0,156.95,0.0
727703,362097,2024-07-31,1.0,173.66,0.0


Number of unique companies in the raw global return data

In [896]:
global_returns_df["gvkey"].nunique()

14896

**Remarks**

*Notice that across both the NA and Global return data, we make up almost 17k 
out the ~19k companies in the emissions data*

Determine the number of companies that are in both the NA and Global datasets

In [897]:
na_returns_uniq_gvkey = na_returns_df["gvkey"].unique()
global_returns_uniq_gvkey = global_returns_df["gvkey"].unique()

na_returns_uniq_gvkey[
    np.isin(na_returns_uniq_gvkey, global_returns_uniq_gvkey)
].size

972

### NA - Data Preparation

In [898]:
na_returns_df

,gvkey,iid,datadate,conm,ajexm,curcdm,prccm,trfm,cshom
0,100001,01,2022-02-28,ACCOR SA,1.0,USD,35.01,NaN,2.618560e+08
1,100001,01,2022-03-31,ACCOR SA,NaN,USD,NaN,1.0000,2.618560e+08
2,100001,01,2022-06-30,ACCOR SA,1.0,USD,26.45,NaN,2.629960e+08
3,100001,01,2022-07-31,ACCOR SA,NaN,USD,NaN,1.0000,2.629960e+08
4,100001,01,2023-07-31,ACCOR SA,1.0,USD,37.90,NaN,2.648940e+08
...,...,...,...,...,...,...,...,...,...
65256,353444,90,2024-04-30,HALEON PLC,1.0,USD,8.53,1.0253,9.132301e+09
65257,353444,90,2024-05-31,HALEON PLC,1.0,USD,8.44,1.0253,9.132301e+09
65258,353444,90,2024-06-30,HALEON PLC,1.0,USD,8.26,1.0253,9.132301e+09
65259,353444,90,2024-07-31,HALEON PLC,1.0,USD,9.15,1.0253,9.132301e+09


**Add year-month**

Isolate the year-month from the data point dates, as the data points 
correspond to montly returns, and we are only interested in this 
level of granularity.

In [899]:
na_returns_df["data_ym"] = na_returns_df["datadate"].dt.to_period('M')

na_returns_df

,gvkey,iid,datadate,conm,ajexm,curcdm,prccm,trfm,cshom,data_ym
0,100001,01,2022-02-28,ACCOR SA,1.0,USD,35.01,NaN,2.618560e+08,2022-02
1,100001,01,2022-03-31,ACCOR SA,NaN,USD,NaN,1.0000,2.618560e+08,2022-03
2,100001,01,2022-06-30,ACCOR SA,1.0,USD,26.45,NaN,2.629960e+08,2022-06
3,100001,01,2022-07-31,ACCOR SA,NaN,USD,NaN,1.0000,2.629960e+08,2022-07
4,100001,01,2023-07-31,ACCOR SA,1.0,USD,37.90,NaN,2.648940e+08,2023-07
...,...,...,...,...,...,...,...,...,...,...
65256,353444,90,2024-04-30,HALEON PLC,1.0,USD,8.53,1.0253,9.132301e+09,2024-04
65257,353444,90,2024-05-31,HALEON PLC,1.0,USD,8.44,1.0253,9.132301e+09,2024-05
65258,353444,90,2024-06-30,HALEON PLC,1.0,USD,8.26,1.0253,9.132301e+09,2024-06
65259,353444,90,2024-07-31,HALEON PLC,1.0,USD,9.15,1.0253,9.132301e+09,2024-07


Missing values must be addressed - by imputating where possible first, and then 
finally dropping rows (which have null values).
- For imputation, we must determine which variables have missing values and 
decide on the appropriate corrective action for each.

In [901]:
print("-- Missing Values:")
na_returns_df.isnull().sum()

-- Missing Values:


gvkey          0
iid            0
datadate       0
conm           0
ajexm        610
curcdm       785
prccm       1225
trfm         942
cshom       2446
data_ym        0
dtype: int64

We can see there are missing values in the close prices, adjustment factors and total return factors.

**Imputation**

In [856]:
""" TODO: nothing right now, but can look into what imputation is possible later """

""" if prices and 
adjustment factors are enough to calculate returns, as long as the total return 
factor is all null or all non-null for each issue. """

' TODO: nothing right now, but can look into what imputation is possible later '

**Drop Rows**

In [902]:
na_returns_df = na_returns_df.dropna()

na_returns_df

,gvkey,iid,datadate,conm,ajexm,curcdm,prccm,trfm,cshom,data_ym
6,100001,91,2021-12-31,ACCOR SA,1.0,USD,6.471,2.1735,2.618560e+08,2021-12
7,100001,91,2022-01-31,ACCOR SA,1.0,USD,7.255,2.1735,2.618560e+08,2022-01
8,100001,91,2022-02-28,ACCOR SA,1.0,USD,6.872,2.1735,2.618560e+08,2022-02
9,100001,91,2022-03-31,ACCOR SA,1.0,USD,6.505,2.1735,2.618560e+08,2022-03
10,100001,91,2022-04-30,ACCOR SA,1.0,USD,6.653,2.1735,2.618560e+08,2022-04
...,...,...,...,...,...,...,...,...,...,...
65255,353444,90,2024-03-31,HALEON PLC,1.0,USD,8.490,1.0253,9.132301e+09,2024-03
65256,353444,90,2024-04-30,HALEON PLC,1.0,USD,8.530,1.0253,9.132301e+09,2024-04
65257,353444,90,2024-05-31,HALEON PLC,1.0,USD,8.440,1.0253,9.132301e+09,2024-05
65258,353444,90,2024-06-30,HALEON PLC,1.0,USD,8.260,1.0253,9.132301e+09,2024-06


Check if the issues have distinct data for each year-month now.

In [903]:
na_returns_issue_ym_entries = na_returns_df.groupby(["gvkey", "iid", "data_ym"]).size()

na_returns_issue_ym_entries

gvkey   iid  data_ym
100001  91   2021-12    1
             2022-01    1
             2022-02    1
             2022-03    1
             2022-04    1
                       ..
353444  90   2024-03    1
             2024-04    1
             2024-05    1
             2024-06    1
             2024-07    1
Length: 62212, dtype: int64

In [904]:
na_returns_issue_ym_entries[
    na_returns_issue_ym_entries != 1
]

Series([], dtype: int64)

The empty series return indicates that this is indeed the case.

**Reindex & Sort**
- by `gvkey`, `iid` and then `data_ym` 
- we know `data_ym` is particularly appropriate 
for indexing now due to its uniqueness within security issues

In [905]:
na_returns_df = na_returns_df.set_index(["gvkey", "iid", "data_ym"])
na_returns_df = na_returns_df.sort_index()

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom  
gvkey  iid data_ym                
100001 91  2021-12  2.618560e+08  
           2022-01  2.618560e+08  
           2022-02  2.618560e+08  
           2022-03  2.618560e+08  
           2022-04  2.618560e+08  
...                          ...  
353444 90  2024-03  9.132301e+09  
           2024-04  9.132301e+09  
           2024-05  9.132301e+09  
           2024-06  9.132301e+09  
           2024-07  9.132301e+09  

[62212 rows x 7 columns]

**Calculate returns.**
- Utilising the year-month entries of each security issue

Check the number of year-month data records for each issue

In [912]:
na_returns_issue_yms = na_returns_df.reset_index(
).groupby(
    ["gvkey", "iid"]
).agg(
    {
        "data_ym": lambda vals: vals.count()
    }
)

na_returns_issue_yms

data_ym
gvkey  iid         
100001 91        32
100012 01        32
100013 90        32
100022 01         2
       02        16
...             ...
350952 90        31
351491 01        31
351590 01        18
       90        32
353444 90        25

[2172 rows x 1 columns]

We can see there are 2204 issues from <1900 companies with varying numbers of 
data points.

In [913]:
na_returns_df.xs((100022, "01"), level=("gvkey", "iid"))

,datadate,conm,ajexm,curcdm,prccm,trfm,cshom
data_ym,,,,,,,
2022-07,2022-07-31,BAYER MOTOREN WERKE AG,1.0,USD,73.8,1.0,59404000.0
2024-02,2024-02-29,BAYER MOTOREN WERKE AG,1.0,USD,109.5,1.0,60844000.0


We also observe that the year-month entries of security issues can be 
non-contiguous when they are incomplete.

In [917]:
na_returns_max_issue_yms = na_returns_issue_yms["data_ym"].max()
print(f"Maximum issue data points: {na_returns_max_issue_yms}")

na_returns_full_issue = na_returns_issue_yms[
    na_returns_issue_yms["data_ym"] == na_returns_max_issue_yms
]

na_returns_full_issue

Maximum issue data points: 32


,,data_ym
gvkey,iid,
100001,91,32
100012,01,32
100013,90,32
100022,90,32
100045,90,32
...,...,...
347328,90,32
347708,90,32
349485,90,32


We observe there are fewer issues with the maximum number of data points.

In [918]:
na_returns_df = na_returns_df.copy()
na_returns_df = na_returns_df.sort_index()

Calculate returns as per the Compustat manual. To do this we first need 
adjusted close.

In [922]:
na_returns_df["adjclose"] = (na_returns_df["prccm"] / na_returns_df["ajexm"]) * na_returns_df["trfm"]

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret  
gvkey  iid data_ym                                   
100001 91  2021-12        NaN        NaN        NaN  
           2022-01   0.121156        0.0   0.008941  
           2022-02  -0.052791        0.0   0.000522  
           2022-03  -0.053405        0.0   0.020464  
           2022-04   0.022752        0.0   0.047160  
...                       ...        ...        ...  
353444 90  2024-03   0.004163        0.0   0.000238  
           2024-04   0.004711        0.0   0.008383  
           2024-05  -0.010551        0.0  -0.015640  
           2024-06  -0.021327        0.0   0.006805  
           2024-07   0.107748        0.0  -0.015809  

[62212 rows x 13 columns]

Exchange rates to the home currency (USD) are joined to allow computation of FX 
returns - that form a part of the overall return.

In [920]:
na_returns_df = na_returns_df.reset_index().merge(
    m_exrts, 
    how="left", 
    left_on=["curcdm", "data_ym"], 
    right_index=True, 
).set_index(
    ["gvkey", "iid", "data_ym"]
)

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  
gvkey  iid data_ym                                                       
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0  
           2022-01  2.618560e+08  15.768743      0.745101           1.0  
           2022-02  2.618560e+08  14.936292      0.745490           1.0  
           2022-03  2.618560e+08  14.138618      0.760746           1.0  
           2022-04  2.618560e+08  14.460296      0.796622           1.0  
...                          ...        ...           ...           ...  
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  
           2024-04  9.132301e+09   8.745809      0.798403           1.0  
           2024-05  9.132301e+09   8.653532      0.785916           1.0  
           2024-06  9.132301e+09   8.468978      0.791264           1.0  
           2024-07  9.132301e+09   9.381495      0.778756           1.0  

[62212 rows x 10 columns]

From the adjusted closes and exchange rates, we can now compute the local and FX returns.

In [921]:
na_returns_df = pd.concat(
    [
        na_returns_df, 
        na_returns_df.groupby(
            level=["gvkey", "iid"]
        )[["adjclose", "exratd_toUSD", "exratd_toGBP"]].pct_change(
        ).rename(
            columns={
                "adjclose": "local_ret", 
                "exratd_toUSD": "USD_fxret", 
                "exratd_toGBP": "GBP_fxret", 
            }
        )
    ], 
    axis=1
)

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret  
gvkey  iid data_ym                                   
100001 91  2021-12        NaN        NaN        NaN  
           2022-01   0.121156        0.0   0.008941  
           2022-02  -0.052791        0.0   0.000522  
           2022-03  -0.053405        0.0   0.020464  
           2022-04   0.022752        0.0   0.047160  
...                       ...        ...        ...  
353444 90  2024-03   0.004163        0.0   0.000238  
           2024-04   0.004711        0.0   0.008383  
           2024-05  -0.010551        0.0  -0.015640  
           2024-06  -0.021327        0.0   0.006805  
           2024-07   0.107748        0.0  -0.015809  

[62212 rows x 13 columns]

Combine local and FX returns to obtain USD returns. Note that these are 
raw returns with varying frequencies.

In [923]:
na_returns_df["USD_ret"] = (
    (1 + na_returns_df["local_ret"]) * (1 + na_returns_df["USD_fxret"]) - 1
)
na_returns_df["GBP_ret"] = (
    (1 + na_returns_df["local_ret"]) * (1 + na_returns_df["GBP_fxret"]) - 1
)

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret   USD_ret   GBP_ret  
gvkey  iid data_ym                                                       
100001 91  2021-12        NaN        NaN        NaN       NaN       NaN  
           2022-01   0.121156        0.0   0.008941  0.121156  0.131180  
           2022-02  -0.052791        0.0   0.000522 -0.052791 -0.052297  
           2022-03  -0.053405        0.0   0.020464 -0.053405 -0.034034  
           2022-04   0.022752        0.0   0.047160  0.022752  0.070985  
...                       ...        ...        ...       ...       ...  
353444 90  2024-03   0.004163        0.0   0.000238  0.004163  0.004401  
           2024-04   0.004711        0.0   0.008383  0.004711  0.013134  
           2024-05  -0.010551        0.0  -0.015640 -0.010551 -0.026026  
           2024-06  -0.021327        0.0   0.006805 -0.021327 -0.014667  
           2024-07   0.107748        0.0  -0.015809  0.107748  0.090236  

[62212 rows x 15 columns]

The differences in months of these raw returns can be used to standardise them 
to a common frequency (monthly).

In [924]:
""" Differences in months, that can be used to standardise returns to the 
same frequency - monthly """
na_returns_df["ret_mfreq"] = na_returns_df.reset_index(
).groupby(
    ["gvkey", "iid"]
)["data_ym"].diff(
).apply(
    lambda x: x.n if pd.notnull(x) else np.nan
).values

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret   USD_ret   GBP_ret  \
gvkey  iid data_ym                                                        
100001 91  2021-12        NaN        NaN        NaN       NaN       NaN   
           2022-01   0.121156        0.0   0.008941  0.121156  0.131180   
           2022-02  -0.052791        0.0   0.000522 -0.052791 -0.052297   
           2022-03  -0.053405        0.0   0.020464 -0.053405 -0.034034   
           2022-04   0.022752        0.0   0.047160  0.022752  0.070985   
...                       ...        ...        ...       ...       ...   
353444 90  2024-03   0.004163        0.0   0.000238  0.004163  0.004401   
           2024-04   0.004711        0.0   0.008383  0.004711  0.013134   
           2024-05  -0.010551        0.0  -0.015640 -0.010551 -0.026026   
           2024-06  -0.021327        0.0   0.006805 -0.021327 -0.014667   
           2024-07   0.107748        0.0  -0.015809  0.107748  0.090236   

                    ret_mfreq  
gvkey  iid data_ym             
100001 91  2021-12        NaN  
           2022-01        1.0  
           2022-02        1.0  
           2022-03        1.0  
           2022-04        1.0  
...                       ...  
353444 90  2024-03        1.0  
           2024-04        1.0  
           2024-05        1.0  
           2024-06        1.0  
           2024-07        1.0  

[62212 rows x 16 columns]

*Sanity check - there should be 0 return month frequencies that 
are non-null for the first year-month entries of issues*

In [925]:
pd.notnull(na_returns_df.groupby(
    level=["gvkey", "iid"]
).agg(
    {
        "ret_mfreq": lambda vals: vals.iloc[0]
    }
)).sum()

ret_mfreq    0
dtype: int64

Standardise the raw USD returns to the monthly frequency.

In [926]:
na_returns_df["m_USD_ret"] = (
    (1 + na_returns_df["USD_ret"]) ** (1/na_returns_df["ret_mfreq"])
) - 1

na_returns_df["m_GBP_ret"] = (
    (1 + na_returns_df["GBP_ret"]) ** (1/na_returns_df["ret_mfreq"])
) - 1

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret   USD_ret   GBP_ret  \
gvkey  iid data_ym                                                        
100001 91  2021-12        NaN        NaN        NaN       NaN       NaN   
           2022-01   0.121156        0.0   0.008941  0.121156  0.131180   
           2022-02  -0.052791        0.0   0.000522 -0.052791 -0.052297   
           2022-03  -0.053405        0.0   0.020464 -0.053405 -0.034034   
           2022-04   0.022752        0.0   0.047160  0.022752  0.070985   
...                       ...        ...        ...       ...       ...   
353444 90  2024-03   0.004163        0.0   0.000238  0.004163  0.004401   
           2024-04   0.004711        0.0   0.008383  0.004711  0.013134   
           2024-05  -0.010551        0.0  -0.015640 -0.010551 -0.026026   
           2024-06  -0.021327        0.0   0.006805 -0.021327 -0.014667   
           2024-07   0.107748        0.0  -0.015809  0.107748  0.090236   

                    ret_mfreq  m_USD_ret  m_GBP_ret  
gvkey  iid data_ym                                   
100001 91  2021-12        NaN        NaN        NaN  
           2022-01        1.0   0.121156   0.131180  
           2022-02        1.0  -0.052791  -0.052297  
           2022-03        1.0  -0.053405  -0.034034  
           2022-04        1.0   0.022752   0.070985  
...                       ...        ...        ...  
353444 90  2024-03        1.0   0.004163   0.004401  
           2024-04        1.0   0.004711   0.013134  
           2024-05        1.0  -0.010551  -0.026026  
           2024-06        1.0  -0.021327  -0.014667  
           2024-07        1.0   0.107748   0.090236  

[62212 rows x 18 columns]

*Notice that there are companies that have gone bankrupt (0 raw close price & 
0 adjusted close price), which leads to further null returns beyond just the 
first month for some companies (due to division by 0).*

In [927]:
na_returns_df[na_returns_df["prccm"] == 0]

,,,datadate,conm,ajexm,curcdm,prccm,trfm,cshom,adjclose,exratd_toGBP,exratd_toUSD,local_ret,USD_fxret,GBP_fxret,USD_ret,GBP_ret,ret_mfreq,m_USD_ret,m_GBP_ret
gvkey,iid,data_ym,,,,,,,,,,,,,,,,,,
175533,01,2024-07,2024-07-31,INFINITY PHARMACEUTICALS INC,1.0,USD,0.0,1.0,90761000.0,0.0,0.778756,1.0,-1.0,0.0,-0.009812,-1.0,-1.0,6.0,-1.0,-1.0


In [928]:
na_returns_df.xs((175533, "01"))

,datadate,conm,ajexm,curcdm,prccm,trfm,cshom,adjclose,exratd_toGBP,exratd_toUSD,local_ret,USD_fxret,GBP_fxret,USD_ret,GBP_ret,ret_mfreq,m_USD_ret,m_GBP_ret
data_ym,,,,,,,,,,,,,,,,,,
2021-12,2021-12-31,INFINITY PHARMACEUTICALS INC,1.0,USD,2.2500,1.0,89015000.0,2.2500,0.738498,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01,2022-01-31,INFINITY PHARMACEUTICALS INC,1.0,USD,1.2400,1.0,89015000.0,1.2400,0.745101,1.0,-0.448889,0.0,0.008941,-0.448889,-0.443961,1.0,-0.448889,-0.443961
2022-02,2022-02-28,INFINITY PHARMACEUTICALS INC,1.0,USD,1.1000,1.0,89015000.0,1.1000,0.745490,1.0,-0.112903,0.0,0.000522,-0.112903,-0.112440,1.0,-0.112903,-0.112440
2022-03,2022-03-31,INFINITY PHARMACEUTICALS INC,1.0,USD,1.1400,1.0,89015000.0,1.1400,0.760746,1.0,0.036364,0.0,0.020464,0.036364,0.057572,1.0,0.036364,0.057572
2022-04,2022-04-30,INFINITY PHARMACEUTICALS INC,1.0,USD,0.7892,1.0,89155000.0,0.7892,0.796622,1.0,-0.307719,0.0,0.047160,-0.307719,-0.275071,1.0,-0.307719,-0.275071
2022-05,2022-05-31,INFINITY PHARMACEUTICALS INC,1.0,USD,0.6724,1.0,89155000.0,0.6724,0.793714,1.0,-0.147998,0.0,-0.003651,-0.147998,-0.151109,1.0,-0.147998,-0.151109
2022-06,2022-06-30,INFINITY PHARMACEUTICALS INC,1.0,USD,0.6323,1.0,89155000.0,0.6323,0.822978,1.0,-0.059637,0.0,0.036869,-0.059637,-0.024967,1.0,-0.059637,-0.024967
2022-07,2022-07-31,INFINITY PHARMACEUTICALS INC,1.0,USD,0.6357,1.0,89155000.0,0.6357,0.821895,1.0,0.005377,0.0,-0.001315,0.005377,0.004055,1.0,0.005377,0.004055
2022-08,2022-08-31,INFINITY PHARMACEUTICALS INC,1.0,USD,1.6100,1.0,89278000.0,1.6100,0.859107,1.0,1.532641,0.0,0.045275,1.532641,1.647306,1.0,1.532641,1.647306


Drop the first month records for which returns could not be calculated. And also 
any other null return records (e.g. from bankrupt companies).

In [929]:
na_returns_df = na_returns_df.dropna()

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
           2022-05 2022-05-31    ACCOR SA    1.0    USD  6.550  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
           2022-05  2.618560e+08  14.236425      0.793714           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret   USD_ret   GBP_ret  \
gvkey  iid data_ym                                                        
100001 91  2022-01   0.121156        0.0   0.008941  0.121156  0.131180   
           2022-02  -0.052791        0.0   0.000522 -0.052791 -0.052297   
           2022-03  -0.053405        0.0   0.020464 -0.053405 -0.034034   
           2022-04   0.022752        0.0   0.047160  0.022752  0.070985   
           2022-05  -0.015482        0.0  -0.003651 -0.015482 -0.019076   
...                       ...        ...        ...       ...       ...   
353444 90  2024-03   0.004163        0.0   0.000238  0.004163  0.004401   
           2024-04   0.004711        0.0   0.008383  0.004711  0.013134   
           2024-05  -0.010551        0.0  -0.015640 -0.010551 -0.026026   
           2024-06  -0.021327        0.0   0.006805 -0.021327 -0.014667   
           2024-07   0.107748        0.0  -0.015809  0.107748  0.090236   

                    ret_mfreq  m_USD_ret  m_GBP_ret  
gvkey  iid data_ym                                   
100001 91  2022-01        1.0   0.121156   0.131180  
           2022-02        1.0  -0.052791  -0.052297  
           2022-03        1.0  -0.053405  -0.034034  
           2022-04        1.0   0.022752   0.070985  
           2022-05        1.0  -0.015482  -0.019076  
...                       ...        ...        ...  
353444 90  2024-03        1.0   0.004163   0.004401  
           2024-04        1.0   0.004711   0.013134  
           2024-05        1.0  -0.010551  -0.026026  
           2024-06        1.0  -0.021327  -0.014667  
           2024-07        1.0   0.107748   0.090236  

[60040 rows x 18 columns]

In [930]:
na_returns_df = na_returns_df.copy()
na_returns_df = na_returns_df.sort_index()

At this point, the computation of monthly returns has been completed.
- w.r.t base currencies of USD and GBP.

**Calculate monthly market capitalisation**
- standardise the currency (USD)

In [931]:
na_returns_df["local_mktval"] = na_returns_df["cshom"] * na_returns_df["prccm"]

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
           2022-05 2022-05-31    ACCOR SA    1.0    USD  6.550  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
           2022-05  2.618560e+08  14.236425      0.793714           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret   USD_ret   GBP_ret  \
gvkey  iid data_ym                                                        
100001 91  2022-01   0.121156        0.0   0.008941  0.121156  0.131180   
           2022-02  -0.052791        0.0   0.000522 -0.052791 -0.052297   
           2022-03  -0.053405        0.0   0.020464 -0.053405 -0.034034   
           2022-04   0.022752        0.0   0.047160  0.022752  0.070985   
           2022-05  -0.015482        0.0  -0.003651 -0.015482 -0.019076   
...                       ...        ...        ...       ...       ...   
353444 90  2024-03   0.004163        0.0   0.000238  0.004163  0.004401   
           2024-04   0.004711        0.0   0.008383  0.004711  0.013134   
           2024-05  -0.010551        0.0  -0.015640 -0.010551 -0.026026   
           2024-06  -0.021327        0.0   0.006805 -0.021327 -0.014667   
           2024-07   0.107748        0.0  -0.015809  0.107748  0.090236   

                    ret_mfreq  m_USD_ret  m_GBP_ret  local_mktval  
gvkey  iid data_ym                                                 
100001 91  2022-01        1.0   0.121156   0.131180  1.899765e+09  
           2022-02        1.0  -0.052791  -0.052297  1.799474e+09  
           2022-03        1.0  -0.053405  -0.034034  1.703373e+09  
           2022-04        1.0   0.022752   0.070985  1.742128e+09  
           2022-05        1.0  -0.015482  -0.019076  1.715157e+09  
...                       ...        ...        ...           ...  
353444 90  2024-03        1.0   0.004163   0.004401  7.753324e+10  
           2024-04        1.0   0.004711   0.013134  7.789853e+10  
           2024-05        1.0  -0.010551  -0.026026  7.707662e+10  
           2024-06        1.0  -0.021327  -0.014667  7.543281e+10  
           2024-07        1.0   0.107748   0.090236  8.356055e+10  

[60040 rows x 19 columns]

In [932]:
na_returns_df["USD_mktval"] = na_returns_df["local_mktval"] * na_returns_df["exratd_toUSD"]
na_returns_df["GBP_mktval"] = na_returns_df["local_mktval"] * na_returns_df["exratd_toGBP"]

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
           2022-05 2022-05-31    ACCOR SA    1.0    USD  6.550  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  ...  \
gvkey  iid data_ym                                                       ...   
100001 91  2022-01  2.618560e+08  15.768743      0.745101           1.0  ...   
           2022-02  2.618560e+08  14.936292      0.745490           1.0  ...   
           2022-03  2.618560e+08  14.138618      0.760746           1.0  ...   
           2022-04  2.618560e+08  14.460296      0.796622           1.0  ...   
           2022-05  2.618560e+08  14.236425      0.793714           1.0  ...   
...                          ...        ...           ...           ...  ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  ...   
           2024-04  9.132301e+09   8.745809      0.798403           1.0  ...   
           2024-05  9.132301e+09   8.653532      0.785916           1.0  ...   
           2024-06  9.132301e+09   8.468978      0.791264           1.0  ...   
           2024-07  9.132301e+09   9.381495      0.778756           1.0  ...   

                    USD_fxret  GBP_fxret   USD_ret   GBP_ret  ret_mfreq  \
gvkey  iid data_ym                                                        
100001 91  2022-01        0.0   0.008941  0.121156  0.131180        1.0   
           2022-02        0.0   0.000522 -0.052791 -0.052297        1.0   
           2022-03        0.0   0.020464 -0.053405 -0.034034        1.0   
           2022-04        0.0   0.047160  0.022752  0.070985        1.0   
           2022-05        0.0  -0.003651 -0.015482 -0.019076        1.0   
...                       ...        ...       ...       ...        ...   
353444 90  2024-03        0.0   0.000238  0.004163  0.004401        1.0   
           2024-04        0.0   0.008383  0.004711  0.013134        1.0   
           2024-05        0.0  -0.015640 -0.010551 -0.026026        1.0   
           2024-06        0.0   0.006805 -0.021327 -0.014667        1.0   
           2024-07        0.0  -0.015809  0.107748  0.090236        1.0   

                    m_USD_ret  m_GBP_ret  local_mktval    USD_mktval  \
gvkey  iid data_ym                                                     
100001 91  2022-01   0.121156   0.131180  1.899765e+09  1.899765e+09   
           2022-02  -0.052791  -0.052297  1.799474e+09  1.799474e+09   
           2022-03  -0.053405  -0.034034  1.703373e+09  1.703373e+09   
           2022-04   0.022752   0.070985  1.742128e+09  1.742128e+09   
           2022-05  -0.015482  -0.019076  1.715157e+09  1.715157e+09   
...                       ...        ...           ...           ...   
353444 90  2024-03   0.004163   0.004401  7.753324e+10  7.753324e+10   
           2024-04   0.004711   0.013134  7.789853e+10  7.789853e+10   
           2024-05  -0.010551  -0.026026  7.707662e+10  7.707662e+10   
           2024-06  -0.021327  -0.014667  7.543281e+10  7.543281e+10   
           2024-07   0.107748   0.090236  8.356055e+10  8.356055e+10   

                      GBP_mktval  
gvkey  iid data_ym                
100001 91  2022-01  1.415517e+09  

## Fundamentals Data

### NA - Loading

In [578]:
fundamentals_df = pd.read_csv(
    "phase1_fundamentals.csv"
)

fundamentals_df

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,curncd,currtr,apdedate,at,ceq,oiadp,revt,costat,mkvalt
0,100080,2021-12-31,2021,INDL,C,D,STD,BAYER AG,USD,EUR,1.137326,2021-12-31,136753.182,37554.496,8084.110,50134.455,A,NaN
1,100080,2022-12-31,2022,INDL,C,D,STD,BAYER AG,USD,EUR,1.068742,2022-12-31,133461.325,41438.343,9863.422,54226.913,A,NaN
2,100080,2023-12-31,2023,INDL,C,D,STD,BAYER AG,USD,EUR,1.105345,2023-12-31,128506.347,36395.707,7641.253,52655.337,A,NaN
3,100091,2021-12-31,2021,INDL,C,D,STD,RENTOKIL INITIAL PLC,USD,GBP,1.354100,2021-12-31,5854.000,1712.000,497.000,4004.000,A,NaN
4,100091,2022-12-31,2022,INDL,C,D,STD,RENTOKIL INITIAL PLC,USD,GBP,1.204900,2022-12-31,14365.000,4939.000,605.000,4475.000,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148,351590,2022-12-31,2022,INDL,C,D,STD,DAIMLER TRUCK HOLDING AG,USD,EUR,1.068742,2022-12-31,68366.372,21430.419,3730.979,54322.030,A,NaN
4149,351590,2023-12-31,2023,INDL,C,D,STD,DAIMLER TRUCK HOLDING AG,USD,EUR,1.105345,2023-12-31,78713.854,23880.987,5579.784,61777.753,A,NaN
4150,353444,2021-12-31,2021,INDL,C,D,STD,HALEON PLC,USD,GBP,1.354100,2021-12-31,46650.099,35687.306,2816.529,12924.885,A,NaN
4151,353444,2022-12-31,2022,INDL,C,D,STD,HALEON PLC,USD,GBP,1.204900,2022-12-31,41948.594,19677.222,2913.448,13082.804,A,NaN


Null values out of the 4153 rows:

In [668]:
print("-- Null values:")
fundamentals_df.isnull().sum()

-- Null values:


gvkey          0
datadate       0
fyear          0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd          0
curncd         0
currtr         0
apdedate     352
at             8
ceq           23
oiadp        640
revt           8
costat         0
mkvalt      1028
dtype: int64

### NA - Data Preparation

## Index Data

In [310]:
index_df = pd.read_csv("phase1_index.csv")

index_df

,gvkeyx,prccm,datadate,conm,tic
0,150918,3379.0814,2022-01-31,S&P Global 1200 Index,I6UNK112
1,150918,3283.8722,2022-02-28,S&P Global 1200 Index,I6UNK112
2,150918,3361.5970,2022-03-31,S&P Global 1200 Index,I6UNK112
3,150918,3089.8620,2022-04-30,S&P Global 1200 Index,I6UNK112
4,150918,3097.8104,2022-05-31,S&P Global 1200 Index,I6UNK112
5,150918,2827.3566,2022-06-30,S&P Global 1200 Index,I6UNK112
6,150918,3032.1650,2022-07-31,S&P Global 1200 Index,I6UNK112
7,150918,2899.3630,2022-08-31,S&P Global 1200 Index,I6UNK112
8,150918,2619.9194,2022-09-30,S&P Global 1200 Index,I6UNK112
9,150918,2795.4613,2022-10-31,S&P Global 1200 Index,I6UNK112


In [10]:
""" Now determine the number of unique companies in the 
different years
"""
print(f"All years: {np.unique(df['fiscalyear'])}")

for year in np.unique(df['fiscalyear']):
    print(f"{year} => {np.unique(df[df['fiscalyear'] == year]['gvkey']).size}")

All years: [2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
 2016 2017 2018 2019 2020 2021 2022 2023]
2002 => 1763
2003 => 2000
2004 => 2885
2005 => 3880
2006 => 4170
2007 => 4307
2008 => 4269
2009 => 4563
2010 => 4723
2011 => 4833
2012 => 4868
2013 => 5757
2014 => 6154
2015 => 6235
2016 => 13882
2017 => 14786
2018 => 16979
2019 => 17378
2020 => 23353
2021 => 23386
2022 => 24053
2023 => 9725


In [32]:
# unique non-nan in returns
returns_gvkeys = np.unique(returns_df[~np.isnan(returns_df["gvkey"])]["gvkey"])

print(returns_gvkeys.size)

24110


In [33]:
env_ret_common_gvkeys = np.intersect1d(
    env_gvkeys, 
    returns_gvkeys, 
    assume_unique=True
)

print(env_ret_common_gvkeys.size)

24110


In [34]:
""" Check the missing gvkeys to see what country they are from """
missing_gvkeys = np.setdiff1d(env_gvkeys, env_ret_common_gvkeys, assume_unique=True)

print(missing_gvkeys.size)

7400


In [77]:
""" Create representatives and break down their distribution """
df_missing_reprs_idx = [(df["gvkey"] == gvkey).idxmax() for gvkey in missing_gvkeys]
missing_dist = df.iloc[df_missing_reprs_idx][["gvkey", "country"]].groupby(
    "country"
).count().reset_index().sort_values('gvkey', ascending=False)

In [78]:
missing_dist

,country,gvkey
79,United States,5220
10,Canada,803
78,United Kingdom,271
13,China,113
24,France,111
...,...,...
58,Peru,1
57,Panama,1
56,Pakistan,1
48,Marshall Islands,1


In [79]:
missing_dist[missing_dist["country"] != "United States"]["gvkey"].sum()

2180

## Checks on Duplicates

In [15]:
company_dups = df[df.duplicated('gvkey', keep=False) == True].sort_values(by="gvkey")

In [26]:
last_iid = None
last_fyears = set()
for i, row in company_dups.iterrows():
    current_iid = row["institutionid"]
    current_fyear = row["fiscalyear"]
    if last_iid is not None and current_iid == last_iid:
        if current_fyear in last_fyears:
            raise Exception(f"Fyear clash!, iid: {current_iid}, prev_fyears: {last_fyears}, clash: {current_fyear}")
        else:
            last_fyears.add(current_fyear)
    else:
        last_iid = current_iid
        last_fyears.clear()
        last_fyears.add(current_fyear)

print("No issues...")

Exception: Fyear clash!, iid: 4415462, prev_fyears: {2022}, clash: 2022

In [31]:
company_dups.head(20)

,periodid,institutionid,reportedcurrencyisocode,tcprimarysectorid,fiscalyear,periodenddate,di_319380,di_319381,di_319382,di_319383,...,streetaddress3,streetaddress4,zipcode,yearfounded,monthfounded,dayfounded,officephonevalue,otherphonevalue,officefaxvalue,webpage
5,30D218CF-2E2A-46B4-AF72-CADF593290E8,4074603,USD,713A00,2022,01/01/2023,0.285,0.021,2.079,0.153,...,NaN,NaN,76011,1961.0,NaN,NaN,972 595 5000,NaN,NaN,www.sixflags.com
2503,1989BCA5-218F-4857-A3CE-1F2114D67F8E,4074603,USD,713A00,2023,31/12/2023,0.302,0.021,2.098,0.147,...,NaN,NaN,76011,1961.0,NaN,NaN,972 595 5000,NaN,NaN,www.sixflags.com
4988,5B249039-D1E2-43D2-825B-DDD233FE2FE0,4996548,USD,561300,2023,31/12/2023,0.015,0.005,0.125,0.040,...,NaN,NaN,75024,2007.0,NaN,NaN,972 692 2400,NaN,NaN,bgsf.com
40,5F205052-5493-41C8-9C8C-1E0FE1FC5DC0,4996548,USD,561300,2022,01/01/2023,0.014,0.005,0.122,0.041,...,NaN,NaN,75024,2007.0,NaN,NaN,972 692 2400,NaN,NaN,bgsf.com
1,01C53196-7DD9-42C7-9D3B-F152BFB3A364,4054841,USD,445000A,2022,01/01/2023,3.061,0.003,186.279,0.203,...,NaN,NaN,1506 MA,1867.0,NaN,NaN,31 88 659 9111,NaN,NaN,www.aholddelhaize.com
2462,793C84DE-3E9D-4CE0-8768-647BA85F9272,4054841,USD,445000A,2023,31/12/2023,3.183,0.003,181.084,0.189,...,NaN,NaN,1506 MA,1867.0,NaN,NaN,31 88 659 9111,NaN,NaN,www.aholddelhaize.com
110,A706AF20-7252-4C95-AA58-2481363942C5,10175068,USD,722000,2022,02/01/2023,0.095,0.053,0.696,0.389,...,NaN,NaN,33309,2011.0,NaN,NaN,954-618-2000,NaN,NaN,www.burgerfi.com
6071,96731D0E-B576-41E4-8386-DAD27811DF60,10175068,USD,722000,2023,01/01/2024,0.092,0.054,0.625,0.368,...,NaN,NaN,33309,2011.0,NaN,NaN,954-618-2000,NaN,NaN,www.burgerfi.com
99,0446986D-7007-4B72-AA5B-49AA5ADB4CF2,28295169,USD,541512,2022,01/01/2023,0.042,0.020,0.362,0.170,...,NaN,NaN,55425,2016.0,NaN,NaN,952 851 5200,NaN,NaN,www.skywatertechnology.com
6011,9AABC54A-2E82-4C6A-99E2-489EF08AE342,28295169,USD,541512,2023,31/12/2023,0.051,0.018,0.420,0.147,...,NaN,NaN,55425,2016.0,NaN,NaN,952 851 5200,NaN,NaN,www.skywatertechnology.com


In [32]:
company_dups[company_dups["institutionid"] == 4415462]

,periodid,institutionid,reportedcurrencyisocode,tcprimarysectorid,fiscalyear,periodenddate,di_319380,di_319381,di_319382,di_319383,...,streetaddress3,streetaddress4,zipcode,yearfounded,monthfounded,dayfounded,officephonevalue,otherphonevalue,officefaxvalue,webpage
23,5282F3CE-9CFA-4830-9116-3B36EFDE5089,4415462,USD,52A000,2022,01/01/2023,0.001,0.001,0.054,0.037,...,NaN,NaN,050059,1993.0,NaN,NaN,7 727 244 5484,NaN,7 727 244 5480,www.homecredit.kz
24,5282F3CE-9CFA-4830-9116-3B36EFDE5089,4415462,USD,52A000,2022,01/01/2023,0.001,0.001,0.054,0.037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


array(['United States', 'Netherlands', 'United Kingdom', 'Canada',
       'Kazakhstan', 'Belarus', 'Australia', 'Belgium', 'Austria',
       'Finland', 'Ireland', nan, 'Singapore', 'France', 'Denmark',
       'Japan', 'Israel', 'Italy', 'South Africa', 'Thailand', 'Germany',
       'China', 'Hong Kong', 'Luxembourg', 'India', 'Switzerland',
       'Malaysia', 'South Korea', 'Kenya', 'New Zealand', 'Spain',
       'Pakistan', 'Saudi Arabia', 'Sweden', 'British Virgin Islands',
       'Kuwait', 'Turkey', 'Philippines', 'Mauritius', 'Bangladesh',
       'Cayman Islands', 'Botswana', 'Egypt', 'Malta', 'Malawi',
       'Jamaica', 'Bermuda', 'Colombia', 'Mexico', 'Norway', 'Brazil',
       'Bahrain', 'Morocco', 'Indonesia', 'Romania', 'Russia',
       'Ivory Coast', 'Tunisia', 'Greece', 'Vietnam', 'Taiwan', 'Nigeria',
       'Oman', 'Qatar', 'Portugal', 'United Arab Emirates', 'Jersey',
       'Poland', 'Bulgaria', 'Chile', 'Reunion', 'Ghana', 'Monaco',
       'Bahamas', 'Guernsey'], dtype=o